In [1]:
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras_facenet.inception_resnet_v1 import InceptionResNetV1

data_path="/Users/santhoshnarayanan/Documents/Turing/GPU/VGGData/testset"
weights_path="../Models/FACENET/Black/hard_1.h5"

2023-01-24 21:02:12.313242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
celebs  = []
 
for r, d, f in os.walk(data_path): # r=root, d=directories, f = files
    for file in f:
        if ('.jpg' in file):
            exact_path = r + "/" + file
            celebs.append(exact_path)

In [3]:
# create a facenet model
target_shape = (160, 160, 3)
embedding = InceptionResNetV1(input_shape=target_shape,
                      classes=512,
                      dropout_keep_prob=0.8,
                      weights_path=None)

embedding.load_weights(weights_path, by_name=True)

# summarize input and output shape
print('Inputs: %s' % embedding.inputs)
print('Outputs: %s' % embedding.outputs)
target_shape = (160, 160)

Metal device set to: AMD Radeon Pro 5500M


2023-01-24 21:02:15.182026: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 21:02:15.182740: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-24 21:02:15.182787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 21:02:15.183094: I tensorflow/core/common_runtime/pluggable_device/plug

Inputs: [<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
Outputs: [<KerasTensor: shape=(None, 512) dtype=float32 (created by layer 'normalize')>]


In [4]:
imgs = [image.load_img(img, target_size=target_shape) for img in celebs]
X = np.float32([(np.float32(img) - 127.5) / 127.5 for img in imgs])
display(X.shape)

(8000, 160, 160, 3)

In [5]:
repA = embedding.predict(X[0:4000], batch_size=64)
display(repA.shape)
repB = embedding.predict(X[4000:8000], batch_size=64)
display(repB.shape)
rep = np.concatenate((repA, repB), axis=0)
display(rep.shape)

2023-01-24 21:02:30.433004: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 12s 139ms/step


(4000, 512)

63/63 [==============================] - 10s 145ms/step


(4000, 512)

(8000, 512)

In [6]:
df = pd.DataFrame(rep)  
df.insert(loc = 0,
          column = 'celeb',
          value = celebs)
display(df)

,celeb,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.081779,-0.058741,0.079340,0.029784,-0.047003,0.008352,-0.036333,0.003825,-0.019637,...,-0.035309,-0.002197,0.032881,-0.016281,-0.053425,0.032875,-0.067107,-0.038267,-0.040752,0.005538
1,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.076732,0.069323,-0.008502,-0.047177,0.012620,-0.036252,0.046890,0.023972,0.061017,...,0.059435,-0.002315,0.020917,0.003658,0.020392,0.020064,-0.042046,-0.078866,-0.022957,0.041304
2,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.001013,-0.019052,0.028401,-0.027400,-0.012141,0.014159,-0.038185,-0.106130,-0.037807,...,-0.014994,0.047262,0.045179,-0.066520,-0.060708,-0.003879,-0.047152,-0.024757,0.065958,-0.038626
3,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.021319,-0.015458,-0.004349,-0.027178,-0.051963,0.096865,-0.023604,-0.077518,-0.041601,...,-0.016574,0.074045,-0.020724,-0.157336,-0.023400,-0.007126,-0.003675,0.037317,0.039764,0.062187
4,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.053552,0.033428,-0.062719,0.003397,-0.045722,0.027397,0.010589,-0.027798,-0.068642,...,0.020975,0.051890,-0.006684,-0.097170,-0.052983,0.014695,-0.009240,-0.067279,0.034687,0.025030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.004696,-0.010367,-0.029232,0.008636,-0.007017,-0.023405,0.009838,-0.011683,-0.011743,...,0.054808,-0.027522,0.067899,0.003342,0.002043,0.073382,0.022101,-0.012498,0.102258,-0.034928
7996,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.038680,-0.020457,-0.008342,-0.011115,-0.029491,0.005607,0.021300,0.058930,-0.042416,...,0.001605,-0.046185,-0.087229,-0.000089,0.035382,-0.002764,-0.014705,-0.003053,-0.101929,0.037225
7997,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.010075,0.038818,0.049824,0.046735,0.067838,0.042641,0.033172,-0.066766,-0.010790,...,-0.003920,-0.023208,0.053816,0.009180,0.061160,-0.003138,0.011354,-0.007068,0.012920,-0.003608
7998,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.036379,0.011102,-0.010189,-0.004699,-0.012314,-0.041049,0.002782,0.037709,-0.050655,...,-0.032225,-0.027505,0.014365,0.031199,0.037425,0.009078,-0.069519,-0.022524,0.027105,-0.075184


In [7]:
df.to_csv('../Results/FACENET/Black/rep.csv')